In [227]:
%reload_ext sql
%sql mysql://iasamori@localhost/?charset=utf8mb4
%config SqlMagic.displaylimit = 1000
%sql SHOW DATABASES;

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

9 rows affected.

Database
campione
dla_tutorial
dlatk_lexica
iasamori
iasamori_album
iasamori_test
iasamori_world
information_schema
sql_intro


In [228]:
%sql USE iasamori;

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

++
||
++
++

### Extract LWIC2015 dictionaries (by message_id)

In [4]:
!dlatkInterface.py --corpdb iasamori \
    --corptable cmt7_cln \
    --correl_field message_id \
    --group_freq_thresh 5 \
    --weighted_lexicon \
    --add_lex_table -l LIWC2015




-----
DLATK Interface Initiated: 2025-06-06 05:08:46
-----
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_cln' AND column_name='message_id'
SQL QUERY: DROP TABLE IF EXISTS feat$cat_LIWC2015_w$cmt7_cln$message_id$1gra
SQL QUERY: CREATE TABLE feat$cat_LIWC2015_w$cmt7_cln$message_id$1gra ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id double, feat VARCHAR(12) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin, value INTEGER, group_norm DOUBLE, KEY `correl_field` (`group_id`), KEY `feature` (`feat`)) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
WORD TABLE feat$1gram$cmt7_cln$message_id
SQL QUERY: ALTER TABLE feat$cat_LIWC2015_w$cmt7_cln$message_id$1gra DISABLE KEYS
5695 out of 569581 group Id's processed; 0.01 complete
11390 out of 569581 group Id's processed; 0.02 complete
17085 out of 569581 group Id's processed; 0.03 complete
22780 out of 569581 group Id's processed; 0.04 complete
28475 out of 5695

In [5]:
a = %sql SHOW TABLES
print(a)

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

12 rows affected.

+----------------------------------------------+
|              Tables_in_iasamori              |
+----------------------------------------------+
|                     cmt7                     |
|                   cmt7_cln                   |
|                   cmt7_en                    |
|                cmt7_en_dedup                 |
|        feat$1gram$cmt7_cln$message_id        |
|    feat$1gram$cmt7_cln$message_id$0_0001     |
|        feat$1gram$cmt7_cln$month_year        |
|     feat$1gram$cmt7_en_dedup$message_id      |
| feat$cat_LIWC2015_w$cmt7_cln$message_id$1gra |
|     feat$meta_1gram$cmt7_cln$message_id      |
|     feat$meta_1gram$cmt7_cln$month_year      |
|   feat$meta_1gram$cmt7_en_dedup$message_id   |
+----------------------------------------------+


### Run Topic Modelling. 200 topics were modelled

In [7]:
num_stopwords = 100
num_topics = 200
alpha = 2
lexicon_database = 'iasamori'

# name your topics something meaningful -- good to encode: source, number, alpha, stopwords
# the f means it's an f-string (can format variables into a string)
# lexicon_name = f'topics7_{num_topics}_a{alpha}s{num_stopwords}'
lexicon_name = f"tpc7_{num_topics}_a{alpha}s{num_stopwords}"

database = 'iasamori'
msgs_table = 'cmt7_cln'
feat_1gram_message = 'feat$1gram$cmt7_cln$message_id'

OUTPUT_FOLDER = './cmt7_cln_lda'
!mkdir -p {OUTPUT_FOLDER}

In [8]:
!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --group_freq_thresh 5 \
    --feat_table '{feat_1gram_message}' \
    --estimate_lda_topics \
    --num_stopwords {num_stopwords} \
    --num_topics {num_topics} \
    --lda_alpha {alpha} \
    --lexicondb {lexicon_database} \
    --save_lda_files {OUTPUT_FOLDER} \
    --lda_lexicon_name {lexicon_name} \
    --mallet_path ./mallet-2.0.8/bin/mallet




-----
DLATK Interface Initiated: 2025-06-06 05:15:05
-----
Writing joined feature lines file: ./cmt7_cln_lda/lda_msgs.txt.
Yielding values with zeros for 569581 groups * 95744 feats.
Wrote joined features file to: ./cmt7_cln_lda/lda_msgs.txt
Estimating LDA topics. This may take a long time!
Estimating LDA topics using Mallet.
Automatically removed stopwords: {'in', 'keep', 'time', 'from', 'my', 'who', 'good', 'the', 'for', 'or', 'all', '️', 'if', ',', 's', '😂', 'you', '🙏', 't', 'been', 'and', 'what', '❤', "it's", 'up', '&', 'don', '’', 'now', 'thank', 'to', 'his', 'not', 'will', 'was', 'love', '!', 'just', 'this', 'but', 'we', 'drugs', 'she', 'that', '🥰', '?', "i'm", 'one', 'be', 'a', 'sorry', 'of', 'like', 'out', 'got', 'go', '😭', 'me', 'life', 'have', '…', 'i', 'there', 'can', 'get', 'with', 'how', 'they', 'pain', 'as', 'are', 'had', 'would', 'at', 'him', 'why', 'on', 'never', 'them', 'u', 'it', 'no', 'an', 'your', 'people', 'do', '..', '🥺', 'need', 'god', 'he', 'know', 'about', '

### Extracting Topic Scores by message_id

In [9]:
database = "iasamori"
msgs_table = "cmt7_cln"
topics_cp_table = "tpc7_200_a2s100_cp"

In [10]:
!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field message_id \
    --group_freq_thresh 5 \
    --add_lex_table -l {topics_cp_table} \
    --lexicondb {database} \
    --weighted_lexicon




-----
DLATK Interface Initiated: 2025-06-06 05:23:30
-----
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_cln' AND column_name='message_id'
SQL QUERY: DROP TABLE IF EXISTS feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$message_id$1gra
SQL QUERY: CREATE TABLE feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$message_id$1gra ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id double, feat VARCHAR(10) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin, value INTEGER, group_norm DOUBLE, KEY `correl_field` (`group_id`), KEY `feature` (`feat`)) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
WORD TABLE feat$1gram$cmt7_cln$message_id
SQL QUERY: ALTER TABLE feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$message_id$1gra DISABLE KEYS
5695 out of 569581 group Id's processed; 0.01 complete
11390 out of 569581 group Id's processed; 0.02 complete
17085 out of 569581 group Id's processed; 0.03 complete
22780 out of 569581 group Id's processed; 0

### Extracting topic scores by month

In [189]:
!dlatkInterface.py \
    --corpdb {database} \
    --corptable {msgs_table} \
    --correl_field month_year \
    --group_freq_thresh 1 \
    --add_lex_table -l {topics_cp_table} \
    --lexicondb {database} \
    --weighted_lexicon




-----
DLATK Interface Initiated: 2025-06-06 12:17:32
-----
query: SELECT column_type FROM information_schema.columns WHERE table_schema='iasamori' AND table_name='cmt7_cln' AND column_name='month_year'
SQL QUERY: DROP TABLE IF EXISTS feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$month_year$1gra
SQL QUERY: CREATE TABLE feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$month_year$1gra ( id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY, group_id text, feat VARCHAR(10) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin, value INTEGER, group_norm DOUBLE, KEY `correl_field` (`group_id`), KEY `feature` (`feat`)) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin ENGINE=MYISAM
WORD TABLE feat$1gram$cmt7_cln$month_year
SQL QUERY: ALTER TABLE feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$month_year$1gra DISABLE KEYS
1 out of 55 group Id's processed; 0.02 complete
2 out of 55 group Id's processed; 0.04 complete
3 out of 55 group Id's processed; 0.05 complete
4 out of 55 group Id's processed; 0.07 complete
5 out of 55 group Id

In [175]:
a = %sql SHOW TABLES
print(a)

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

19 rows affected.

+--------------------------------------------------------+
|                   Tables_in_iasamori                   |
+--------------------------------------------------------+
|                          cmt7                          |
|                        cmt7_cln                        |
|              cmt7_cln_lda$tpc7_200_a2s100              |
|                     cmt7_cln_lwic                      |
|                     cmt7_cln_tpc7                      |
|                        cmt7_en                         |
|                     cmt7_en_dedup                      |
|             feat$1gram$cmt7_cln$message_id             |
|         feat$1gram$cmt7_cln$message_id$0_0001          |
|             feat$1gram$cmt7_cln$month_year             |
|          feat$1gram$cmt7_en_dedup$message_id           |
|      feat$cat_LIWC2015_w$cmt7_cln$message_id$1gra      |
| feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$message_id$1gra |
| feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$month_year$1gra

### Merge Comments and LWIC/Topics Tables

In [12]:
%load_ext rpy2.ipython

In [13]:
%%R

source("./psych290_data/helper_files/psych290RcodeV1.R")

require(tidyverse)
require(RSQLite)
require(ggthemes)
require(ggpubr)
require(grid)
require(reshape2)
require(psych)
require(lm.beta)
library(caret)

library(DBI)
library(RMySQL)

dbcon <- dbConnect(
  drv = MySQL(),
  host = "localhost",  # or your server address
  dbname = "iasamori",
  user = "iasamori",
  port = 3306
)

R[write to console]: Loading required package: ggthemes

R[write to console]: Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


R[write to console]: Loading required package: RSQLite

R[write to console]: Loading required package: ggpubr

R[write to console]: Loading required package: psych

R[write to console]: 
Attaching package: ‘psych’


R[write to console]: The following objects are masked from ‘package:ggplot2’:

    %+%, alpha


R[write to console]: Loading required package: lm.beta

R[write to console]: Loading required package: lattice

R[write to console]: 
Attaching package: ‘caret’


R[write to console]: The following object is masked from ‘package:purrr’:

    lift




#### Merge Comments and LWIC

In [14]:
%%R

lwic_dict <- dbGetQuery(dbcon, "select * from `feat$cat_LIWC2015_w$cmt7_cln$message_id$1gra`")
lwic_dict_wide <- importFeat(lwic_dict)

# topic_dist <- dbGetQuery(dbcon, "select * from `feat$cat_LIWC2015_w$comments_7_cleaned$message_id$1gra`")

cmt7_cln <- dbGetQuery(dbcon, "select * from `cmt7_cln`")

merged_comments_lwic <- merge(cmt7_cln, lwic_dict_wide, by.x = "message_id", by.y = "group_id", all.x = TRUE)

dbWriteTable(conn = dbcon,
             name = "cmt7_cln_lwic",
             value = merged_comments_lwic,
             overwrite = TRUE,
             append = FALSE,
             row.names = FALSE)

head(merged_comments_lwic)

    message_id                                   message     video_id
1 6.912639e+18                Hi I really love your eyes 6.912577e+18
2 6.912664e+18 Hola btw I love your hair it's so cute 😁 6.912577e+18
3 6.912689e+18          hii 👋🏾😁 how was ur day today? 6.912577e+18
4 6.912689e+18      AAAHHHH your so pretty 🥺Heyyoooo 💙 6.912577e+18
5 6.912702e+18        Hiiii and you are sooooo beautiful 6.912577e+18
6 6.912731e+18         Ello shawty is that good or no 😏 6.912577e+18
  create_time like_count parent_comment_id reply_count month_year _intercept
1  1609474308          1      6.912577e+18           0    12_2020          1
2  1609479909          1      6.912577e+18           0     1_2021          1
3  1609485790          1      6.912577e+18           1     1_2021          1
4  1609485812          1      6.912577e+18           0     1_2021          1
5  1609488828          2      6.912577e+18           1     1_2021          1
6  1609495660          2      6.912577e+18           0 

In [18]:
%%sql

CREATE INDEX idx_cmt7_cln_lwic_message_id ON cmt7_cln_lwic (message_id);
CREATE INDEX idx_cmt7_cln_lwic_create_time ON cmt7_cln_lwic (create_time);
CREATE INDEX idx_cmt7_cln_lwic_month_year ON cmt7_cln_lwic (month_year);

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

++
||
++
++

#### Merge Comments and Topics

In [33]:
%%R

tpc7_dist <- dbGetQuery(dbcon, "select * from `feat$cat_tpc7_200_a2s100_cp_w$cmt7_cln$message_id$1gra`")
tpc7_dist_wide <- importFeat(tpc7_dist)
head(tpc7_dist_wide)
# topic_dist <- dbGetQuery(dbcon, "select * from `feat$cat_LIWC2015_w$comments_7_cleaned$message_id$1gra`")

cmt7_cln <- dbGetQuery(dbcon, "select * from `cmt7_cln`")

merged_comments_tpc7 <- merge(cmt7_cln, tpc7_dist_wide, by.x = "message_id", by.y = "group_id", all.x = TRUE)

dbWriteTable(conn = dbcon,
             name = "cmt7_cln_tpc7",
             value = merged_comments_tpc7,
             overwrite = TRUE,
             append = FALSE,
             row.names = FALSE)

head(merged_comments_tpc7)

    message_id                                   message     video_id
1 6.912639e+18                Hi I really love your eyes 6.912577e+18
2 6.912664e+18 Hola btw I love your hair it's so cute 😁 6.912577e+18
3 6.912689e+18          hii 👋🏾😁 how was ur day today? 6.912577e+18
4 6.912689e+18      AAAHHHH your so pretty 🥺Heyyoooo 💙 6.912577e+18
5 6.912702e+18        Hiiii and you are sooooo beautiful 6.912577e+18
6 6.912731e+18         Ello shawty is that good or no 😏 6.912577e+18
  create_time like_count parent_comment_id reply_count month_year _intercept 0
1  1609474308          1      6.912577e+18           0    12_2020          1 0
2  1609479909          1      6.912577e+18           0     1_2021          1 0
3  1609485790          1      6.912577e+18           1     1_2021          1 0
4  1609485812          1      6.912577e+18           0     1_2021          1 0
5  1609488828          2      6.912577e+18           1     1_2021          1 0
6  1609495660          2      6.912577e+18 

In [34]:
%%sql

CREATE INDEX idx_cmt7_cln_tpc7_message_id ON cmt7_cln_tpc7 (message_id);
CREATE INDEX idx_cmt7_cln_tpc7_create_time ON cmt7_cln_tpc7 (create_time);
CREATE INDEX idx_cmt7_cln_tpc7_month_year ON cmt7_cln_tpc7 (month_year);

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

++
||
++
++

### Generate Topic Worclouds

In [31]:
!dlatkInterface.py \
    --corpdb iasamori \
    --corptable cmt7_cln \
    --correl_field message_id \
    --make_all_topic_wordclouds \
    --tagcloud_colorscheme blue \
    --topic_lexicon {topics_freq_table} \
    --lexicondb {lexicon_database} \
    --output ./cmt7_cln_lda/tpc7_200_a2s100_wc




-----
DLATK Interface Initiated: 2025-06-06 07:36:03
-----
Generating wordclouds using amueller's python package...
Wordcloud created at: ./cmt7_cln_lda/tpc7_200_a2s100_wc_alltopics/topic_0_wc.png
Generating wordclouds using amueller's python package...
Wordcloud created at: ./cmt7_cln_lda/tpc7_200_a2s100_wc_alltopics/topic_1_wc.png
Generating wordclouds using amueller's python package...
Wordcloud created at: ./cmt7_cln_lda/tpc7_200_a2s100_wc_alltopics/topic_10_wc.png
Generating wordclouds using amueller's python package...
Wordcloud created at: ./cmt7_cln_lda/tpc7_200_a2s100_wc_alltopics/topic_100_wc.png
Generating wordclouds using amueller's python package...
Wordcloud created at: ./cmt7_cln_lda/tpc7_200_a2s100_wc_alltopics/topic_101_wc.png
Generating wordclouds using amueller's python package...
Wordcloud created at: ./cmt7_cln_lda/tpc7_200_a2s100_wc_alltopics/topic_102_wc.png
Generating wordclouds using amueller's python package...
Wordcloud created at: ./cmt7_cln_lda/tpc7_200_